# install stuff

In [ ]:
! pip3 install pyspark pandas scikit-learn

# setup pyspark

In [1]:
import pyspark.sql
from pyspark.sql import functions as sf
import pyspark

In [ ]:
sparkConf = pyspark.SparkConf() \
    .set("spark.executor.memory", "512m")\
    .set("spark.driver.memory", "512m")
ss = pyspark.sql.SparkSession.builder.config(conf=sparkConf).master('local[1]').getOrCreate()

# load data

In [ ]:
train_data = ss.read.orc('./hw_data/posts_train.orc')
test_data = ss.read.orc('./hw_data/posts_test.orc')
channel_data = ss.read.orc('./hw_data/channels_orc')

# make features

In [ ]:
def make_features(df):
    cols = ['id']
    if 'views' in df.columns:
        cols.append('views')
    return df.select(*cols)

In [ ]:
train_features = make_features(train_data).cache()
test_features = make_features(test_data).cache()

In [ ]:
train_features.write.csv('train_csv', mode='overwrite', header=True)
test_features.write.csv('test_csv', mode='overwrite', header=True)

# load features to pandas
you also can use .toPandas()

In [ ]:
import subprocess
import glob
import os
import shutil
import pandas as pd

def load_and_merge_csv(path, **kwargs):
    dfs = []
    for g in glob.glob(os.path.join(path, '*.csv')):
        dfs.append(pd.read_csv(g, **kwargs))
    res = pd.concat(dfs)
    res = res.set_index('id')
    return res

In [ ]:
trainXY = load_and_merge_csv_from_hdfs('train_csv')
testX = load_and_merge_csv_from_hdfs('test_csv')

In [ ]:
import numpy as np

Ycol = 'views'
to_drop = ['channel_id']
trainX, trainY = trainXY.drop(Ycol, axis=1).drop(to_drop, axis=1), trainXY[Ycol]
trainY = np.log(trainY + 100)

testX = testX.drop(to_drop, axis=1)

# train your model and predict test

In [ ]:
prediction = np.random.random(testX.shape[0])

In [ ]:
prediction.shape

In [ ]:
assert prediction.shape == (244386,)

# submit

In [ ]:
! curl -H 'Cache-Control: no-cache' https://raw.githubusercontent.com/mike0sv/lsml_submit_server/2024/src/client.py -o client.py

In [ ]:
import client

In [ ]:
client.make_eval(pd.DataFrame({'views': prediction}, index=testX.index))

In [ ]:
client.check_results()